<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [12]:
# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sklearn stuff

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

# ML Stuff

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load dataset

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [13]:
df.shape

(7043, 21)

In [14]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [15]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,0178-SZBHO,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [19]:
# ok lets clean this up

encoder = OneHotEncoder()
label_encoder = LabelEncoder()

binary = ['gender', 'Partner', 'Dependents', 'PhoneService', 
          'PaperlessBilling', 'Churn']
multi = ['MultipleLines', 'InternetService', 'OnlineSecurity',
        'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
        'StreamingMovies', 'Contract', 'PaymentMethod', 'TotalCharges']


In [21]:
for col in binary:
    df[col] = encoder.fit_transform(df[[col]]).todense()
    df[col] = df[col].astype(int)

In [22]:
for col in multi:
    df[col] = label_encoder.fit_transform(df[[col]])
    df[col] = df[col].astype(int)

/home/ivan/anaconda3/envs/Unit4Sprint2/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,2,29.85,2505,1
1,5575-GNVDE,0,0,1,1,34,0,0,0,2,...,2,0,0,0,1,1,3,56.95,1466,1
2,3668-QPYBK,0,0,1,1,2,0,0,0,2,...,0,0,0,0,0,0,3,53.85,157,0
3,7795-CFOCW,0,0,1,1,45,1,1,0,2,...,2,2,0,0,1,1,0,42.30,1400,1
4,9237-HQITU,1,0,1,1,2,0,0,1,0,...,0,0,0,0,0,0,2,70.70,925,0


In [24]:
df.shape

(7043, 21)

In [25]:
# Take out churn for testing and remove useless columns

X = df.drop(columns=['Churn', 'customerID', 'TotalCharges'])
y = df['Churn']

X.shape, y.shape

((7043, 18), (7043,))

In [26]:
# Scale the data

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [30]:
# Ok lets fit a model!

inputs = X.shape[1]
epochs = 20
batch_size = 42

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=inputs))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                608       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(X, y, 
          validation_split=0.25, 
          epochs=epochs, 
          batch_size=batch_size)

W0417 11:37:39.845904 139674187978560 deprecation.py:323] From /home/ivan/anaconda3/envs/Unit4Sprint2/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5282 samples, validate on 1761 samples
Epoch 1/20
5282/5282 [==============================] - 0s 88us/sample - loss: 2.6035 - acc: 0.5369 - val_loss: 1.2136 - val_acc: 0.6735
Epoch 2/20
5282/5282 [==============================] - 0s 74us/sample - loss: 0.9098 - acc: 0.7081 - val_loss: 0.9300 - val_acc: 0.7178
Epoch 3/20
5282/5282 [==============================] - 0s 82us/sample - loss: 0.7375 - acc: 0.7457 - val_loss: 0.7904 - val_acc: 0.7456
Epoch 4/20
5282/5282 [==============================] - 1s 112us/sample - loss: 0.6594 - acc: 0.7478 - val_loss: 0.7181 - val_acc: 0.7428
Epoch 5/20
5282/5282 [==============================] - 1s 117us/sample - loss: 0.6137 - acc: 0.7520 - val_loss: 0.7013 - val_acc: 0.7490
Epoch 6/20
5282/5282 [==============================] - 1s 115us/sample - loss: 0.6082 - acc: 0.7584 - val_loss: 0.7580 - val_acc: 0.7615
Epoch 7/20
5282/5282 [==============================] - 1s 106us/sample - loss: 0.5976 - acc: 0.7658 - val_loss: 0.6255 - val_a

In [32]:
model.evaluate(X, y)

7043/7043 [==============================] - 0s 34us/sample - loss: 0.5236 - acc: 0.7874


[0.5236386538787472, 0.7874485]

In [34]:
# ok now for hyperparameters

# lets try batch size first

params = {'batch_size': [10, 20, 50, 100, 250, 500, 1000],
         'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

/home/ivan/anaconda3/envs/Unit4Sprint2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f0827a82ba8> does not.

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?